In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data  =  pd.read_csv("../input/titanic/train.csv")
df=data.copy()
df

In [ ]:
data2 = pd.read_csv("../input/titanic/test.csv")
df_test=data2.copy()
df_test

In [ ]:
df.isnull().sum()

In [ ]:
df = df[df['Embarked'].notna()]
df.isnull().sum()

In [ ]:
df.dtypes

In [ ]:
# label encoding the data 
"""
from sklearn.preprocessing import LabelEncoder 
  
le = LabelEncoder() 
  
df['Sex']= le.fit_transform(df['Sex'])
df['Sex'].value_counts()
# 1 for male 0 for female
""""

In [ ]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res) 

In [ ]:
features_to_encode = ['Embarked', 'Sex']
for feature in features_to_encode:
    df = encode_and_bind(df, feature)

In [ ]:
df

In [ ]:
df['Survived'].value_counts()
# We can observe there is more death than survival 

In [ ]:
df['Pclass'].value_counts()

In [ ]:
df['Ticket'].value_counts()

In [ ]:
df['Cabin'].value_counts()

In [ ]:
df['Embarked_C'].value_counts()

In [ ]:
df['Embarked_S'].value_counts()

In [ ]:
df['Embarked_Q'].value_counts()

In [ ]:
df['Age'].value_counts()

In [ ]:
df['Fare'].value_counts()

In [ ]:
df['Sex_male'].value_counts()

In [ ]:
df['Sex_female'].value_counts()

In [ ]:
df['Survived'].hist()

In [ ]:
sns.pairplot(df)
#plt.show()

In [ ]:
df.corr(method='pearson')

In [ ]:
df.corr(method='spearman')
##  Corr Survived & Sex  Survived & Pclass and Survived & Fare 
##  Corr Pclass & Age and Pclass & Fare
##  Corr Age SibSp

In [ ]:
#Dropping unecessery tables
df=df.drop(['Cabin','Ticket','Name'],axis=1,errors='ignore')
df

In [ ]:
df=df.dropna()
df

In [ ]:
"""
Logistic Regression
Naive Bayes Classifier
K-Nearest Neighbors
Decision Tree
Random Forest
Support Vector Machines

"""
from sklearn.model_selection import train_test_split

X=df.drop(['PassengerId','Survived','PassengerId'],axis=1,errors='ignore')
y=df['Survived']
X,y

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

In [ ]:
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

In [ ]:
"""
le = LabelEncoder() 
  
X['Sex']= le.fit_transform(X['Sex'])
X['Sex'].value_counts()
"""


In [ ]:
# We do training with X,y
from sklearn import tree
DT_clf = tree.DecisionTreeClassifier()
DT_clf.fit(X_train, y_train)

In [ ]:
DT_train_score= DT_clf.score(X_train, y_train)
print('train score:', DT_train_score)
DT_test_score= DT_clf.score(X_test, y_test)
print('test score:', DT_test_score)

In [ ]:
DT_y_pred = DT_clf.predict(X_test)

In [ ]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, DT_y_pred))
print("Precision:",metrics.precision_score(y_test, DT_y_pred))
print("Recall:",metrics.recall_score(y_test, DT_y_pred))

In [ ]:
# Create Decision Tree classifer object
DT_clf_2 = tree.DecisionTreeClassifier(criterion="entropy", max_depth=3)

# Train Decision Tree Classifer
DT_clf_2.fit(X_train,y_train)

#Predict the response for test dataset
DT_y_pred2 = DT_clf_2.predict(X_test)

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test,DT_y_pred2))
print("Precision:",metrics.precision_score(y_test, DT_y_pred2))
print("Recall:",metrics.recall_score(y_test,DT_y_pred2))

In [ ]:
from sklearn import svm
SVM_clf = svm.SVC(kernel='linear')
SVM_clf.fit(X_train, y_train)
y_pred_svm = SVM_clf.predict(X_test)

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_svm))
print("Precision:",metrics.precision_score(y_test, y_pred_svm))
print("Recall:",metrics.recall_score(y_test, y_pred_svm))

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid_SVM = {#'kernel': ('linear', 'rbf','poly'), 
                  'C':[1, 1.5]}
                  #'epsilon':[0.1,0.2] }#,0.5,0.3]}

In [ ]:
import time

start_time = time.time()
grid_SVM = GridSearchCV(SVM_clf , param_grid_SVM,cv=5)
grid_SVM.fit(X_train, y_train)
SVM_time_ = time.time() - start_time
print("--- %s seconds ---" % (SVM_time_))

In [ ]:
grid_SVM

In [ ]:
print(grid_SVM.best_score_)
print(grid_SVM.best_params_)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RF_clf =  RandomForestClassifier(
    n_estimators=50,
    criterion='gini',
    max_depth=5,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features='auto',
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    bootstrap=True,
    oob_score=False,
    n_jobs=-1,
    random_state=0,
    verbose=0,
    warm_start=False,
    class_weight='balanced')
RF_clf.fit(X_train, y_train)
RF_y_pred = RF_clf.predict(X_test)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
cv = ShuffleSplit(n_splits=10, test_size=0.3, random_state=0)
cross_val_score(RF_clf, X_train, y_train, cv=cv)

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, RF_y_pred))
print("Precision:",metrics.precision_score(y_test, RF_y_pred))
print("Recall:",metrics.recall_score(y_test, RF_y_pred))

In [ ]:
from sklearn.linear_model import LogisticRegression
LR_clf = LogisticRegression(solver='lbfgs', max_iter=1000,random_state=0)
LR_clf.fit(X_train, y_train)
LR_y_pred=LR_clf.predict(X_test)

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, LR_y_pred))
print("Precision:",metrics.precision_score(y_test, LR_y_pred))
print("Recall:",metrics.recall_score(y_test, LR_y_pred))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KN_clf = KNeighborsClassifier(n_neighbors=3)
KN_clf.fit(X_train, y_train)
KN_y_pred=KN_clf.predict(X_test)

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, KN_y_pred))
print("Precision:",metrics.precision_score(y_test, KN_y_pred))
print("Recall:",metrics.recall_score(y_test, KN_y_pred))

In [ ]:
from sklearn.linear_model import SGDClassifier
SGDC_clf = SGDClassifier(max_iter=1000, tol=1e-3)
SGDC_clf.fit(X_train, y_train)
SGDC_y_pred=KN_clf.predict(X_test)

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, SGDC_y_pred))
print("Precision:",metrics.precision_score(y_test, SGDC_y_pred))
print("Recall:",metrics.recall_score(y_test, SGDC_y_pred))

In [ ]:
from sklearn.naive_bayes import GaussianNB
GNB_clf = GaussianNB()
GNB_clf.fit(X_train, y_train)
GNB_y_pred=KN_clf.predict(X_test)

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, GNB_y_pred))
print("Precision:",metrics.precision_score(y_test, GNB_y_pred))
print("Recall:",metrics.recall_score(y_test, GNB_y_pred))

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
LSVC_clf = make_pipeline(StandardScaler(), LinearSVC(max_iter=10000,random_state=0, tol=1e-5))
LSVC_clf.fit(X_train, y_train)
LSVC_y_pred=LSVC_clf.predict(X_test)

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, LSVC_y_pred))
print("Precision:",metrics.precision_score(y_test, LSVC_y_pred))
print("Recall:",metrics.recall_score(y_test, LSVC_y_pred))

In [ ]:
from sklearn.linear_model import Perceptron
PER_clf = Perceptron(tol=1e-3, random_state=0)
PER_clf.fit(X_train, y_train)
PER_y_pred=PER_clf.predict(X_test)


In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, LSVC_y_pred))
print("Precision:",metrics.precision_score(y_test, LSVC_y_pred))
print("Recall:",metrics.recall_score(y_test, LSVC_y_pred))

In [ ]:
# Working of the test data set

In [ ]:
df_test=df_test.drop(['PassengerId','Name','Ticket','Cabin'],axis=1,errors='ignore')
df_test

In [ ]:
""""
le = LabelEncoder() 
  
df_test['Embarked']= le.fit_transform(df_test['Sex'])
df_test['Sex']=le.fit_transform(df_test['Sex'])
df_test

"""

In [ ]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res)

In [ ]:
features_to_encode = ['Embarked', 'Sex']
for feature in features_to_encode:
    df_test= encode_and_bind(df_test, feature)

In [ ]:
df_test

In [ ]:
# Filling Nan values with 0 
df_test=df_test.fillna(0)

In [ ]:
y_pred_final = RF_clf.predict(df_test)
y_pred_final

In [ ]:
# Numpy arry to DataFrame
target = pd.DataFrame(data=y_pred_final,columns=["Survived"])
target

In [ ]:
target.reset_index()
#target[0] = 'PassengerId'
target['PassengerId'] = target.index+892
target

In [ ]:
target= target[['PassengerId','Survived']]
target

In [ ]:
target.to_csv('RF_2.csv',sep=',',encoding='utf-8',index = False, header=True)